In [ ]:
#needs to be run at every colab instance
# !pip install torchinfo
# !pip install torchaudio

In [ ]:
import torchaudio
import torch
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import sys
from torch.autograd import Variable
import math
# from torchsummary import summary
from torchinfo import summary

import os
import soundfile as sf
from os import listdir
from os.path import isdir, join
import pandas as pd
import pathlib
import torch.optim as optim
import tensorflow as tf

from tqdm.notebook import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# device ='cpu'
print(device)

In [ ]:
output_folder =  pathlib.Path('output')

if not output_folder.exists():
    os.mkdir(output_folder)

In [ ]:
data_dir = pathlib.Path('/content/data')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'speech_commands_v0.02.tar.gz',
      origin="http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz",
      extract=True,
      cache_dir='.', cache_subdir='data')

In [ ]:
output_folder =str(output_folder)

In [ ]:
# data_dir = 'C:/Users/cferr/Documents/4th Year/FYP Data/speech_commands_v0.02/'

In [ ]:
keywords = [name for name in listdir(data_dir) if isdir(join(data_dir, name))]
#remove bg noise as it not needed and requires extra pre-processing
keywords.remove('_background_noise_')
print(keywords)

In [ ]:
word2index = {
    # core words
    "backward": 0,
    "bed": 1,
    "bird": 2,
    "cat": 3,
    "dog": 4,
    "down": 5,
    "eight": 6,
    "five": 7,
    "follow": 8,
    "forward": 9,
    "four": 10,
    "go": 11,
    "happy": 12,
    "house": 13,
    "learn": 14,
    "left": 15,
    "marvin": 16,
    "nine": 17,
    "no": 18,
    "off": 19,
    "on":20,
    "one":21,
    "right":22,
    "seven":23,
    "sheila":24,
    "six":25,
    "stop":26,
    "three":27,
    "tree":28,
    "two":29,
    "up":30,
    "visual":31,
    "wow":32,
    "yes":33,
    "zero":34
}

index2word = [word for word in word2index]

In [ ]:
filenames = []
y = []
for word_class in word2index:
    for files in listdir(join(data_dir, word_class)):
        filenames.append(join(word_class, files))
        y.append(word2index[word_class]) 

In [ ]:
#create a dictionary of the filenames and labels
combined_dict = dict(zip(filenames,y))
#save that dictionary
np.save('files_dict', combined_dict)

In [ ]:
from sklearn.model_selection import train_test_split
train_data_initial, test_data, train_classes_initial, validation_classes = train_test_split(filenames, y,
                                                                      test_size=0.2, random_state=42, shuffle=True)

In [ ]:
train_data, validation_data, train_classes, validation_classes =train_test_split(train_data_initial, train_classes_initial,
                                                                      test_size=0.1, random_state=42, shuffle=False)

In [ ]:
def getLists(files_in, output_name):
  MyFile=open(output_name,'w')

  for element in files_in:
      MyFile.write(element)
      MyFile.write('\n')
  MyFile.close()

In [ ]:
getLists(filenames, 'fileslist.scp')
getLists(test_data, 'test.scp')
getLists(train_data, 'train.scp')
getLists(validation_data, 'val.scp')

In [ ]:
data_folder = data_dir
outdim = len(keywords)
lab_dict = np.load('files_dict.npy', allow_pickle=True).item()

lr=0.001
batch_size=128
Batch_dev=128
N_epochs=100
N_batches=595
N_eval_epoch=8
seed=42

torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
def ReadList(list_file):
 f=open(list_file,"r")
 lines=f.readlines()
 list_sig=[]
 for x in lines:
    list_sig.append(x.rstrip())
 f.close()
 return list_sig

In [ ]:
wav_lst_tr=ReadList('train.scp')
snt_tr=len(wav_lst_tr)
# test list
wav_lst_te=ReadList('test.scp')
snt_te=len(wav_lst_te)
#validation list
wav_lst_val=ReadList('val.scp')
snt_val=len(wav_lst_val)

In [ ]:
snt_tr

In [ ]:
wlen=16000

### SincConv module
based on https://github.com/mravanelli/SincNet

In [ ]:
import numpy as np
import math

class SincConv(torch.nn.Module):
    @staticmethod
    def to_mel(hz):
        return 2595 * np.log10(1 + hz / 700)

    @staticmethod
    def to_hz(mel):
        return 700 * (10 ** (mel / 2595) - 1)

    def __init__(self, out_channels, kernel_size, sample_rate=16000, in_channels=1, stride=1, padding=0, min_low_hz=50, min_band_hz=50):
        super(SincConv,self).__init__()

        if in_channels != 1:
            msg = "SincConv only support one input channel (here, in_channels = {%i})" % (in_channels)
            raise ValueError(msg)

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        # Forcing the filters to be odd (i.e, perfectly symmetrics)
        if kernel_size%2==0:
            self.kernel_size=self.kernel_size+1
        # parameters    
        self.stride = stride
        self.padding = padding
        self.sample_rate = sample_rate
        self.min_low_hz = min_low_hz
        self.min_band_hz = min_band_hz
        # initialize filterbanks such that they are equally spaced in Mel scale
        low_hz = 30
        high_hz = self.sample_rate / 2 - (self.min_low_hz + self.min_band_hz)
        mel = np.linspace(self.to_mel(low_hz), self.to_mel(high_hz), self.out_channels + 1)
        hz = self.to_hz(mel)
        # filter lower frequency (out_channels, 1)
        self.low_hz_ = torch.nn.Parameter(torch.Tensor(hz[:-1]).view(-1, 1))
        # self.low_hz_ =torch.Tensor(hz[:-1]).view(-1, 1).to(device)
        # filter frequency band (out_channels, 1)
        self.band_hz_ = torch.nn.Parameter(torch.Tensor(np.diff(hz)).view(-1, 1))
        # self.band_hz_ =torch.Tensor(np.diff(hz)).view(-1, 1).to(device)

        # Hamming window
        n_lin=torch.linspace(0, (self.kernel_size/2)-1, steps=int((self.kernel_size/2))) # computing only half of the window
        self.window_=0.54-0.46*torch.cos(2*math.pi*n_lin/self.kernel_size);
        # (1, kernel_size/2)
        n = (self.kernel_size - 1) / 2.0
        self.n_ = 2*math.pi*torch.arange(-n, 0).view(1, -1) / self.sample_rate # Due to symmetry, I only need half of the time axes

    def forward(self, waveforms):
        self.n_ = self.n_.to(waveforms.device)
        self.window_ = self.window_.to(waveforms.device)
        low = self.min_low_hz  + torch.abs(self.low_hz_)
        # low= low.to(device)
        high = torch.clamp(low + self.min_band_hz + torch.abs(self.band_hz_),self.min_low_hz,self.sample_rate/2)
        # high=high.to(device)
        band=(high-low)[:,0]
        
        # f_times_t_low = torch.matmul(low, self.n_).to(device)
        # f_times_t_high = torch.matmul(high, self.n_).to(device)
        f_times_t_low = torch.matmul(low, self.n_)
        f_times_t_high = torch.matmul(high, self.n_)


        band_pass_left=((torch.sin(f_times_t_high)-torch.sin(f_times_t_low))/(self.n_/2))*self.window_ # Equivalent of Eq.4 of the reference paper (SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET). I just have expanded the sinc and simplified the terms. This way I avoid several useless computations. 
        band_pass_center = 2*band.view(-1,1)
        band_pass_right= torch.flip(band_pass_left,dims=[1])
        
        band_pass=torch.cat([band_pass_left,band_pass_center,band_pass_right],dim=1)
        band_pass = band_pass / (2*band[:,None])

        self.filters = (band_pass).view(self.out_channels, 1, self.kernel_size)
        return torch.nn.functional.conv1d(waveforms, self.filters, stride=self.stride, padding=self.padding, dilation=1, bias=None, groups=1) 

### LogAbs Acitvation Module

In [ ]:
class LogAbs(torch.nn.Module):
    def __init__(self):
        super(LogAbs,self).__init__()

    def forward(self, x):
        return torch.log(torch.abs(x) + 1)

### Sinc and Depthwise Separable Conv Blocks

In [ ]:
def SincConvBlock(c, k, s, sample_rate=16000):
    sinc_conv = SincConv(out_channels=c, kernel_size=k, sample_rate=sample_rate, in_channels=1, stride=s, padding=k//2)
    avg_pool = torch.nn.AvgPool1d(kernel_size=2, stride=2)
    return torch.nn.Sequential(sinc_conv, LogAbs(), torch.nn.BatchNorm1d(c), avg_pool)

def DSConvBlock(c_in, c, k, s):
    depth_conv = torch.nn.Conv1d(in_channels=c_in, out_channels=c_in, kernel_size=k, stride=s, padding=k//2, groups=c_in)
    point_conv = torch.nn.Conv1d(in_channels=c_in, out_channels=c, kernel_size=1, stride=1)
    return torch.nn.Sequential(depth_conv, point_conv, torch.nn.ReLU(), torch.nn.BatchNorm1d(c), torch.nn.AvgPool1d(2), torch.nn.Dropout(0.1))

## Batch generator to extract waveforms 

In [ ]:
def batch_gen(batch_size,data_folder,wav_lst,N_snt,wlen,lab_dict):
    sig_batch=torch.zeros([batch_size,1, wlen])
    lab_batch=torch.zeros(batch_size)
    snt_id_arr=torch.randint(0, N_snt, (batch_size,))
    
    waveforms=[]
    for i in range(batch_size):
        waveform, sample_rate = torchaudio.load(str(data_folder)+'/'+wav_lst[snt_id_arr[i]])

        #ensure all waveforms are padded to a length of 16000
        snt_len=waveform.shape[1]
        if(snt_len < 16000):
            target = torch.zeros((1,16000))
            target[:, :snt_len] = waveform
            waveform=target 
        waveforms.append(waveform)
        lab_batch[i]=lab_dict[wav_lst[snt_id_arr[i]]]
    s = torch.cat(waveforms).unsqueeze(1) # Audio samples, Feature first (1 since mono)
    return s, lab_batch

Test the batch_gen function to ensure its working as intended

In [ ]:
[inp,lab]=batch_gen(3,data_folder,wav_lst_tr,snt_tr,wlen,lab_dict)
inp.to(device)

In [ ]:
print(inp.shape, lab)

In [ ]:
def plot_tensor(x, ar=8):
    for i in range(x.shape[0]):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.imshow(x[i].log2().numpy())
        fig.set_figwidth(100)
        plt.axis('off')
        plt.show()

Obersevation of spectrogram plots produced by the sinc-convolution layer

In [ ]:
sinc_conv_block = SincConvBlock(c=40, k=101, s=8)
sc = sinc_conv_block(inp).unsqueeze(1).detach()
scc = sc.view(sc.size(0), -1)
scc -= scc.min(1, keepdim=True)[0]
scc /= scc.max(1, keepdim=True)[0]
sc = scc.view(3, 1, 40, 1000).squeeze(1)
print(sc.shape)
plot_tensor(sc)

### Full Model. Based on Mittermaier et al.

In [ ]:
sinc_conv_model = torch.nn.Sequential(
    SincConvBlock(c=40, k=101, s=8),
    DSConvBlock(40,160,25,2), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    torch.nn.AvgPool1d(15),
    torch.nn.Flatten(),
    torch.nn.Linear(160,35),
    torch.nn.Softmax(dim=1)
)

In [ ]:
[inp,lab]=batch_gen(1,data_folder,wav_lst_tr,snt_tr,wlen,lab_dict)
sinc_conv_model=sinc_conv_model.to(device)
inp = inp.to(device)
summary(sinc_conv_model, input_data=[inp])

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
model=sinc_conv_model.to(device)

In [ ]:
n = count_parameters(model)
print("Number of parameters: %s" % n)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
LOG_DIR = "experiment_dir"
train_writer = SummaryWriter(os.path.join(LOG_DIR, "train"))
val_writer = SummaryWriter(os.path.join(LOG_DIR, "val"))

In [ ]:
%load_ext tensorboard

In [ ]:
cost = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr, eps=1e-8)


scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
N_batches_val = 66
epochs=60
snt_val

## Training the Model


In [ ]:
min_valid_loss = np.inf
for e in range(epochs):
    train_loss = 0.0
    model.train()     # Optional when not using Model Specific layer
    for i in range(N_batches):
        [inp,target]=batch_gen(batch_size,data_folder,wav_lst_tr,snt_tr,wlen,lab_dict)
        inp = inp.to(device)
        target = target.type(torch.LongTensor)
        target = target.to(device)
        output = model(inp)
        
        optimizer.zero_grad()
        loss = cost(output,target)
        train_writer.add_scalar("Loss/Train", loss, e)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()
    
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for i in range(N_batches_val):
        [inp,target]=batch_gen(batch_size,data_folder,wav_lst_val,snt_val,wlen,lab_dict)
        inp = inp.to(device)
        target = target.type(torch.LongTensor)
        target = target.to(device)

        output = model(inp)
        loss = cost(output,target)
        val_writer.add_scalar("Loss/Val", loss, e)
        valid_loss = loss.item()

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss:.6f} \t\t Validation Loss: {valid_loss:.6f}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'SincConv_model.pth')
    # scheduler.step()

In [ ]:
train_writer.close()
val_writer.close()

In [ ]:
%tensorboard --logdir experiment_dir/

# Testing the Model

In [ ]:
N_batches_te=165

In [ ]:
accuracy=[]
def test(model, epoch):
    samples=0
    model.eval()
    correct = 0
    for i in range(N_batches_te):
        [data,target]=batch_gen(batch_size,data_folder,wav_lst_te,snt_te,wlen,lab_dict)
        data = data.to(device)
        target = target.to(device)

        output = model(data)

        pred = pred=torch.max(output,dim=1)[1]
        
        correct+= (pred == target).float().sum() 
        samples = samples + (1*len(data))
        # update progress bar
        # pbar.update(pbar_update)
    accuracy.append(correct/(batch_size*N_batches_te))
    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{(batch_size*N_batches_te)} ({100. * correct /(batch_size*N_batches_te):.2f}%)\n")
    # print(samples)

In [ ]:
model=sinc_conv_model.to(device)
model.load_state_dict(torch.load('Trained_Models/SincConv_model.pth'))

In [ ]:
for i in range(5):
  test(model, i)

# Removal of Block

In [ ]:
sinc_conv_model_4block = torch.nn.Sequential(
    SincConvBlock(c=40, k=101, s=8),
    DSConvBlock(40,160,25,2), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    torch.nn.AvgPool1d(30),
    torch.nn.Flatten(),
    torch.nn.Linear(160,35),
    torch.nn.Softmax(dim=1)
)

In [ ]:
summary(sinc_conv_model_4block, input_data=[inp])

In [ ]:
sinc_conv_model_4block=sinc_conv_model_4block.to(device)

In [ ]:
n = count_parameters(sinc_conv_model_4block)
print("Number of parameters: %s" % n)

In [ ]:
optimizer = optim.Adam(sinc_conv_model_4block.parameters(), lr=lr, eps=1e-8)

In [ ]:
min_valid_loss = np.inf
for e in range(epochs):
    train_loss = 0.0
    sinc_conv_model_4block.train()     # Optional when not using Model Specific layer
    for i in range(N_batches):
        [inp,target]=batch_gen(batch_size,data_folder,wav_lst_tr,snt_tr,wlen,lab_dict)
        inp = inp.to(device)
        target = target.type(torch.LongTensor)
        target = target.to(device)
        output = sinc_conv_model_4block(inp)
        
        optimizer.zero_grad()
        loss = cost(output,target)
        # train_writer.add_scalar("Loss/Train", loss, e)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()
    
    valid_loss = 0.0
    sinc_conv_model_4block.eval()     # Optional when not using Model Specific layer
    for i in range(N_batches_val):
        [inp,target]=batch_gen(batch_size,data_folder,wav_lst_val,snt_val,wlen,lab_dict)
        inp = inp.to(device)
        target = target.type(torch.LongTensor)
        target = target.to(device)

        output = sinc_conv_model_4block(inp)
        loss = cost(output,target)
        # val_writer.add_scalar("Loss/Val", loss, e)
        valid_loss = loss.item()

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss:.6f} \t\t Validation Loss: {valid_loss:.6f}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(sinc_conv_model_4block.state_dict(), 'SincConvModel4.pth')

## Testing 4 block model

In [ ]:
sinc_conv_model_4block = sinc_conv_model_4block
sinc_conv_model_4block.load_state_dict(torch.load('Trained_Models/SincConvModel4.pth'))

sinc_conv_model_4block.to(device)

In [ ]:
for i in range(5):
  test(sinc_conv_model_4block, i)

# Addition of Block

In [ ]:
sinc_conv_model_6block = torch.nn.Sequential(
    SincConvBlock(c=40, k=101, s=8),
    DSConvBlock(40,160,25,2), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1),
    DSConvBlock(160,160,9,1),
    DSConvBlock(160,160,9,1),   
    torch.nn.AvgPool1d(7),
    torch.nn.Flatten(),
    torch.nn.Linear(160,35),
    torch.nn.Softmax(dim=1)
)

In [ ]:
n = count_parameters(sinc_conv_model_6block)
print("Number of parameters: %s" % n)

# Loading and testing 6 block model

In [ ]:
sinc_conv_model_6block=sinc_conv_model_6block.to(device)

In [ ]:
sinc_conv_model_6block.load_state_dict(torch.load('Trained_Models/6layerSincConv.pth'))


In [ ]:
for i in range(5):
  test(sinc_conv_model_4block, i)

### Tracing 5block model for use on netron.app

In [ ]:
sinc_conv_model.eval()
model_input = (inp)
traced_model = torch.jit.trace(sinc_conv_model, model_input)
print(traced_model)
traced_model.save('traced.pt')